### This notebook should be run on **`Google Colab`** <br>
if you don't have enough performance for heavy computation

In [1]:
import numpy as np
from keras.models import Sequential 
from keras.optimizers import Adam
from keras.layers import Flatten, Dense, Dropout
import pandas as pd
import matplotlib.pyplot as plt
import cv2, random, os, ntpath
from sklearn.utils import shuffle
from keras.layers import Convolution2D #check
from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from imgaug import augmenters as iaa

Using TensorFlow backend.


In [ ]:
!git clone https://github.com/minlaxz/simulator_track

In [ ]:
datadir = 'simulator_track'
columns = ['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed']
data = pd.read_csv(os.path.join(datadir,'driving_log.csv'), names = columns)
pd.set_option('display.max_colwidth', -1)
data.head(3)

In [ ]:
def path_leaf(path):
  head, tail= ntpath.split(path)
  return tail

data['center'] = data['center'].apply(path_leaf)
data['left'] = data['left'].apply(path_leaf)
data['right'] = data['right'].apply(path_leaf)
data.head(3)

In [ ]:
num_bins = 25

samples_per_bin = 250 

hist, bins= np.histogram(data['steering'], num_bins)  #6386 steering points will split into 25 cate
print(len(bins))                                      #bins return from histogram is 26 length but no zero value

center = (bins[:-1] + bins[1:]) * 0.5
print(len(center))                                    #rearraged including zero

plt.bar(center, hist, width=0.05)
#plt.plot((np.min(data['steering']), np.max(data['steering'])), (200,200))
plt.plot((-1,1),(samples_per_bin,samples_per_bin))

In [ ]:
print(bins)
print('total data', len(data))  #6386
remove_list = []
for j in range(num_bins):       #0 -> 25
  list_ = []
  for i in range(len(data['steering'])):   #0 -> 6386
    if data['steering'][i] >= bins[j] and data['steering'][i] <= bins[j+1]:
      list_.append(i)
  list_ = shuffle(list_)          #shuffle for data stablity
  list_ = list_[samples_per_bin:] #250 to the end **CROP
  remove_list.extend(list_)       #like append but whole list adding

print('removed' , len(remove_list))
data.drop(data.index[remove_list], inplace=True)
print('remaining' , len(data))

hist, _ = np.histogram(data['steering'], (num_bins))

In [ ]:
#plt.bar(center, hist, width=0.05)
#plt.plot((np.min(data['steering']), np.max(data['steering'])), (samples_per_bin,samples_per_bin))

In [ ]:
def load_img_steering(datadir, df):
  image_path=[]
  steering = []
  for i in range(len(data)):  #0 -> 1874 dropped data
    indexed_data = data.iloc[i]
    center , left , right = indexed_data[0], indexed_data[1], indexed_data[2]
    image_path.append(os.path.join(datadir+center.strip()))
    steering.append(float(indexed_data[3]))
  image_paths = np.asarray(image_path)
  steerings = np.asarray(steering)
  return image_paths, steerings

image_paths , steerings = load_img_steering(datadir+'/IMG/' , data)

In [ ]:
X_train, X_valid , y_train, y_valid = train_test_split(image_paths, steerings, test_size=0.2, random_state=6)
print('Training samples', len(X_train))
print('Validation samples', len(X_valid))

In [ ]:
def zoom(image):
  zoom = iaa.Affine(scale=(1, 1.3))
  image = zoom.augment_image(image)
  return image
def pan(image):
   pan = iaa.Affine(translate_percent={"x":(-0.1, 0.1), "y":(-0.1,0.1)})
   image = pan.augment_image(image)
   return image
def img_random_brightness(image):
  brightness = iaa.Multiply((0.2, 1.2))
  image = brightness.augment_image(image)
  return image
def img_random_flip(image,steering_angle):
  image = cv2.flip(image, 1)
  steering_angle = -steering_angle
  return image, steering_angle
def random_augment(image, steering_angle):
    #image = mpimg.imread(image)
    if np.random.rand() < 0.5:
      image = pan(image)
    if np.random.rand() < 0.5:
      image = zoom(image)
    if np.random.rand() < 0.5:
      image = img_random_brightness(image)
    if np.random.rand() < 0.5:
      image, steering_angle = img_random_flip(image, steering_angle)
    
    return image, steering_angle

In [ ]:
def img_preprocess(img):
    img = img[60:135,:,:]
    img = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
    img = cv2.GaussianBlur(img,  (3, 3), 0)
    img = cv2.resize(img, (200, 66))
    img = img/255
    return img

In [ ]:
def batch_generator(image_paths, steering_ang, batch_size, istraining):
  while True:
    batch_img = []
    batch_steering = []
    for i in range(batch_size):
      random_index = random.randint(0,len(image_paths)-1)

      if istraining:
        im, steering = random_augment(image_paths[random_index], steering_ang[random_index])
      else:
        im = mpimg.imread(image_paths[random_index])
        steering = steering_ang[random_index]
      
      im = img_preprocess(im)
      batch_img.append(im)
      batch_steering.append(steering)
    yield (np.asarray(batch_img), np.asarray(batch_steering))

In [ ]:
def nvidia_model():
  model = Sequential()
  model.add(Convolution2D(24, 5, 5, subsample=(2,2), input_shape=(66,200,3) , activation='elu' ))
  model.add(Convolution2D(36, 5 ,5, subsample=(2,2), activation='elu'))
  model.add(Convolution2D(48, 5 ,5, subsample=(2,2), activation='elu'))
  model.add(Convolution2D(64, 3 ,3, activation='elu'))
  model.add(Convolution2D(64, 3 ,3, activation='elu'))
  model.add(Dropout(0.5))

  model.add(Flatten())

  model.add(Dense(100, activation='elu'))
  model.add(Dropout(0.5))

  model.add(Dense(50, activation='elu'))
  model.add(Dropout(0.5))

  model.add(Dense(10, activation='elu'))
  model.add(Dropout(0.5))
  
  model.add(Dense(1))

  optimizer = Adam(lr = 0.001)
  model.compile(loss='mse', optimizer=optimizer)
  
  return model

In [ ]:
model = nvidia_model()
print(model.summary())

In [ ]:
history = model.fit_generator(batch_generator(X_train, y_train, 100, 1 ),
                              steps_per_epoch=300,
                              epochs=10,
                              validation_data=batch_generator(X_valid, y_valid, 100, 0),
                              validation_steps=200,
                              verbose=1,
                              shuffle=1)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.title('Loss')
plt.xlabel('epoch')